In [5]:
import pandas as pd
from translate.storage.tmx import tmxfile
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Data available at https://opus.nlpl.eu/OpenSubtitles-v2018.php
# https://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/tmx/en-pt.tmx.gz
with open("../../data/en-pt.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'pt')

In [ ]:
# put tmx file into a pandas dataframe
df = pd.DataFrame(columns=['source', 'target'])
for unit in tmx_file.units:
    df = df.append({'source': unit.source, 'target': unit.target}, ignore_index=True)
df

In [8]:
df

,source,target
0,Pyke to plumber command.,Pyke para o Comando dos Encanadores.
1,"Current location, sector zed niner.",Localização actual: sector 09.
2,Approaching location of unidentified spacecraft.,A aproximar-me da localização da nave não iden...
3,"Copy that, pyke.","Entendido, Pyke."
4,Do you have visual?,Consegues vê-la?
...,...,...
933561,"If I had another life, I'd be a nun.","Se eu voltar a nascer, irei para freira."
933562,"- Grandpa, give me the case!","- Avô, dá-me a mala."
933563,"- No, I won't.","- Não, não ta dou! Mau!"
933564,"You're hurting me, Umberto!","Estás a magoar-me, Umberto!"


In [ ]:
# Save it to save time
df.to_csv("../../data/corpus/en-pt.tsv", index=False, sep='\t')